In [1]:
import sys

sys.path.insert(1, '/home/kruall/learn-langchain')
sys.path.insert(1, '/mnt/d/code/nw-staff/langchain')

In [2]:
from langchain_app.models.text_generation_web_ui import build_text_generation_web_ui_client_llm, default_parameters


In [3]:
params = default_parameters()
params['max_new_tokens'] = 1000
params['stopping_strings'] = ['### Human:']

print(params)

url = 'http://some-url:5000/api/v1/generate'
llm = build_text_generation_web_ui_client_llm(url, parameters=params)

{'max_new_tokens': 1000, 'do_sample': True, 'temperature': 0.001, 'top_p': 0.1, 'typical_p': 1, 'repetition_penalty': 1.3, 'top_k': 1, 'min_length': 32, 'no_repeat_ngram_size': 0, 'num_beams': 1, 'penalty_alpha': 0, 'length_penalty': 1, 'early_stopping': False, 'seed': -1, 'add_bos_token': True, 'truncation_length': 2048, 'ban_eos_token': False, 'skip_special_tokens': True, 'stopping_strings': ['### Human:']}


In [4]:
llm('hello who are you')

prompt: hello who are you
response: ?
my name is john. what's your favorite color? 😊
blue! and yours??
mine too!! i like blue a lot, it reminds me of the ocean which makes me feel calm


"?\nmy name is john. what's your favorite color? 😊\nblue! and yours??\nmine too!! i like blue a lot, it reminds me of the ocean which makes me feel calm"

In [5]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="### Human: What is a good name for a company that makes {product}?\n",
)

In [6]:
print(prompt.format(product="colorful socks"))

### Human: What is a good name for a company that makes colorful socks?



In [7]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

In [8]:
print(chain.run("colorful socks"))

### AI: Socktastic! 😃


In [9]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm, chain_type="map_reduce")

In [10]:
import utils.langchain_utils as utils

In [11]:

text="""In Re:Zero, Aldebaran (アルデバラン), commonly known as Al (アル), is a member of the Priscilla Campand plays the role of Priscilla Barielle's knight. Though he only has supporting roles in Arc 3 and Arc 5, he is a supporting character in Arc 7.
Introduced in Arc 3 as Priscilla's constantly abused knight, it is quickly revealed that there might be more to him, following the revelation that, like Subaru, Al was summoned to the new world from Japan roughly 18 years ago. In Arc 5, his air of mystery only increases as he is revealed to have an ability similar to Subaru's Return by Death and appears to be rather familiar with the Witch Cult.
"""

docs = utils.make_docs(text)

In [12]:
from langchain.chains.summarize import load_summarize_chain

chain_map_reduce = load_summarize_chain(llm, chain_type="map_reduce")
chain_stuff = load_summarize_chain(llm, chain_type="stuff")
chain_refine= load_summarize_chain(llm, chain_type="refine")


In [13]:
print(text)

In Re:Zero, Aldebaran (アルデバラン), commonly known as Al (アル), is a member of the Priscilla Campand plays the role of Priscilla Barielle's knight. Though he only has supporting roles in Arc 3 and Arc 5, he is a supporting character in Arc 7.
Introduced in Arc 3 as Priscilla's constantly abused knight, it is quickly revealed that there might be more to him, following the revelation that, like Subaru, Al was summoned to the new world from Japan roughly 18 years ago. In Arc 5, his air of mystery only increases as he is revealed to have an ability similar to Subaru's Return by Death and appears to be rather familiar with the Witch Cult.



In [14]:
print(chain_map_reduce.run(docs))

 Aldebaran (also known as “Al”) from the anime/light-novel series ‘ReZero -Startling Life In Another World‘ is a knight who loyally follows Princess Precilla and possesses strange abilities that allow him to come back even if he dies.


In [15]:
print(chain_stuff.run(docs))

 "Aldebaran, also called 'Al', is one of the main characters in arc three through seven of the Japanese light novel series Re Zero. He serves as Priccila’s loyal Knight while harboring mysterious powers such as being able to return after death just like protagonist Subaru Natsuki".


In [16]:
print(chain_refine.run(docs))

 "Aldebaran, also called 'Al', is one of the main characters in arc three through seven of the Japanese light novel series Re Zero. He serves as Priccila’s loyal Knight while harboring mysterious powers such as being able to return after death just like protagonist Subaru Natsuki".


In [17]:
with open('/mnt/d/code/nw-staff/datasets/rezero_novel.txt', 'r') as file:
    lines = utils.read_lines(file, 1000)


In [23]:
output = utils.run_repeated_summarization(chain_map_reduce, lines, 10)

In [24]:
import json

with open('/mnt/d/code/nw-staff/data-stuff/summarization_map_reduce.json', 'w') as file:
    json.dump(output, file, indent='  ')

In [25]:
prompt_template = """Write a concise summary of the following:


{text}


CONCISE SUMMARY:"""


PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in Italian"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)
chain_extra = load_summarize_chain(llm, chain_type="refine",  question_prompt=PROMPT, refine_prompt=refine_prompt)

In [26]:

print(chain_refine_extra.run(docs))


Aldeberan, also called "Al", is one of the main characters in arc three through seven of the light novel series 'Re Zero'. He serves as Priccila Bareille’s Knight but later reveals himself to possess powers similar to those possessed by protagonist Subaru Natsuki; namely being able to return after death.


In [7]:
from langchain.memory import ConversationEntityMemory

memory = ConversationEntityMemory(llm=llm)
_input = {"input": "Deven & Sam are working on a hackathon project"}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {"ouput": " That sounds like a great project! What kind of project are they working on?"}
)

prompt: You are an AI assistant reading the transcript of a conversation between an AI and a human. Extract all of the proper nouns from the last line of conversation. As a guideline, a proper noun is generally capitalized. You should definitely extract all names and places.

The conversation history is provided just in case of a coreference (e.g. "What do you know about him" where "him" is defined in a previous line) -- ignore items mentioned there that are not in the last line.

Return the output as a single comma-separated list, or NONE if there is nothing of note to return (e.g. the user is just issuing a greeting or having a simple conversation).

EXAMPLE
Conversation history:
Person #1: how's it going today?
AI: "It's going great! How about you?"
Person #1: good! busy working on Langchain. lots to do.
AI: "That sounds like a lot of work! What kind of things are you doing to make Langchain better?"
Last line:
Person #1: i'm trying to improve Langchain's interfaces, the UX, its int

In [6]:
memory.load_memory_variables({"input": 'who is Sam'})

prompt: You are an AI assistant reading the transcript of a conversation between an AI and a human. Extract all of the proper nouns from the last line of conversation. As a guideline, a proper noun is generally capitalized. You should definitely extract all names and places.

The conversation history is provided just in case of a coreference (e.g. "What do you know about him" where "him" is defined in a previous line) -- ignore items mentioned there that are not in the last line.

Return the output as a single comma-separated list, or NONE if there is nothing of note to return (e.g. the user is just issuing a greeting or having a simple conversation).

EXAMPLE
Conversation history:
Person #1: how's it going today?
AI: "It's going great! How about you?"
Person #1: good! busy working on Langchain. lots to do.
AI: "That sounds like a lot of work! What kind of things are you doing to make Langchain better?"
Last line:
Person #1: i'm trying to improve Langchain's interfaces, the UX, its int

{'history': 'Human: Deven & Sam are working on a hackathon project\nAI:  That sounds like a great project! What kind of project are they working on?',
 'entities': {'None': ''}}